In [2]:
!gdown 1w7Riq_itzcijqkvSW0yyivvlw0IgRGa_
!unzip Dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1w7Riq_itzcijqkvSW0yyivvlw0IgRGa_
To: /home/chienbinhronggreen/Study/Project/Face-based-Employee-Check-In-System/Dataset.zip
100%|██████████████████████████████████████| 1.34M/1.34M [00:00<00:00, 3.07MB/s]
Archive:  Dataset.zip
replace Dataset/Avatar_Aaron_Eckhart.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
import os

In [2]:
dataset_path = "Dataset"
os.listdir(dataset_path)   # Liet ke cac file va thu muc

['Avatar_Anil_Ramsook.jpg',
 'Avatar_Andrew_Cuomo.jpg',
 'Avatar_Camille_Lewis.jpg',
 'Avatar_Thang_Duong.jpg',
 'Avatar_Amy_Pascal.jpg',
 'Avatar_Andrew_Bernard.jpg',
 'Avatar_Tien_Huy.jpg',
 'Avatar_Phuc_Thinh.JPG',
 'Avatar_Carla_Gay_Balingit.jpg',
 'Avatar_Dang_Nha.jpg',
 'Avatar_Amy_Redford.jpg',
 'Avatar_Hoang_Nguyen.jpg',
 'Avatar_Minh_Chau.jpg',
 'Avatar_Anh_Khoi.JPG',
 'Avatar_Aaron_Guiel.jpg',
 'Avatar_Thuan_Duong.jpg',
 'Avatar_Quoc_Thai.JPG',
 'Avatar_Aaron_Eckhart.jpg']

In [3]:
image_paths = []
labels = []

for filename in os.listdir(dataset_path):
    if filename.endswith(('jpg', 'JPG', 'jpeg', 'png')):
        image_paths.append(os.path.join(dataset_path, filename))
        fileNameWithoutTail = filename.split('.')[0]
        labels.append(fileNameWithoutTail[7:])

In [4]:
IMAGE_SIZE = 300

from PIL import Image
import numpy as np

def convert_image_to_vector(image_path):
    image = Image.open(image_path).resize((IMAGE_SIZE, IMAGE_SIZE))
    img = np.array(image)

    # Check anh den trang
    if img.shape == 2:
        img = np.stack((img, ) * 3, axis=-1)
    
    # Chuan hoa ve doan [0, 1]
    vector = img.astype('float32') / 255.0

    return vector.flatten()

In [5]:
import faiss

VECTOR_DIM = IMAGE_SIZE * IMAGE_SIZE * 3

index = faiss.IndexFlatL2(VECTOR_DIM)
label_map = []

for idx in range(len(image_paths)):
    vector = convert_image_to_vector(image_paths[idx])
    index.add(np.array([vector]))
    label_map.append(labels[idx])

faiss.write_index(index, "employee_images.index")
np.save("label_map.npy", label_map)

In [8]:
def search_similar_images(query_image_path, k = 5):
    index = faiss.read_index("employee_images.index")
    label_map = np.load("label_map.npy")

    query_vector = convert_image_to_vector(query_image_path)
    distances, indices = index.search(np.array([query_vector]), k)

    results = []
    for i in range(len(indices[0])):
        employee_name = label_map[indices[0][i]]
        distance = distances[0][i]
        
        results.append((employee_name, distance))

    return results

In [9]:
query_image = "Dataset/Avatar_Phuc_Thinh.JPG"
print(search_similar_images(query_image))

[('Phuc_Thinh', 0.0), ('Dang_Nha', 22539.059), ('Quoc_Thai', 24997.75), ('Amy_Pascal', 25327.293), ('Anh_Khoi', 27042.953)]
